<h1> Analysis </h1>

## Imports

In [3]:
import numpy as np
import h5py
import os
import sys
import glob
import tkinter as tk
from tkinter import filedialog

root = tk.Tk()
root.withdraw()

''

## Reference and blast analysis

#### Count the number of single references

In [4]:
# check if all ids and all sequences are unique
# for 3XR6 dataset
def check_ids_seqs(referenceFile):
    # assert it is a fasta file
    assert referenceFile[-5:] == "fasta"
    # if full in the reference name, headers are present
    cropped = True
    if "full" in referenceFile:
        cropped = False
    #dictionaries to keep track of the count of unqiue id and sequence
    unique_id = {}
    unique_seq = {}
    total_count = 0
    with open(referenceFile, 'r') as f:
        for index, line in enumerate(f):
            if index%2==0:
                assert line[0] == '>'
                line = line.strip("\n").strip()
                ref_id = int(line[9:]) #only the id not the "bc" part in 3XR6 dataset
                if(ref_id not in unique_id):
                    unique_id[ref_id] = 1
                else:
                    unique_id[ref_id] += 1
            else:
                assert line[0] in ['A', 'C', 'T', 'G']
                seq = line.strip("\n").strip()
                if not cropped:
                    seq = seq[25:50]
                if(seq not in unique_seq):
                    unique_seq[seq] = 1
                else:
                    unique_seq[seq] += 1
                total_count += 1
    abnormality_detected = False
    # check duplicates for ids
    it = 1
    missing_list = []
    for key_id in unique_id:
        if(unique_id[key_id]>1):
            print("Id " + key_id + " has: " + str(unique_id[key_id]) + "occurences")
            abnormality_detected = True
        if(key_id != it):
            for i in range(it, key_id):
                missing_list.append(i)
            it = key_id + 1
        else:
            it +=1
    print("Missing ids: " + str(missing_list))
    # check duplicates for sequences
    for seq_key in unique_seq:
        if(unique_seq[seq_key]>1):
            print("Sequence " + seq_key + " has: " + str(unique_seq[seq_key]) + "occurences")
            abnormality_detected = True
    # In the case nothing abnormal was found        
    if not abnormality_detected:
        print("All sequences and ids are unique")




In [97]:
referenceFile = filedialog.askopenfilename()
check_ids_seqs(referenceFile)

Missing ids: [721, 812, 3698]
All sequences and ids are unique


In [15]:
# count the number of references in a blastn file
def count_ids_f5(fast5Folder):
    id_count = {}
    filenames = glob.glob(fast5Folder + "/*.fast5")
    for fast5File in filenames:
        # try to open fast5 file
        if not fast5File.endswith('fast5'):
            print("Not a fast5 file")
            return
        try:
            fast5_data = h5py.File(fast5File, 'r')
        except IOError:
            print('Error opening file')
            return

        read_list = list(fast5_data.items())
        for i in range(len(read_list)):
            readstep = (read_list[i])
            read = readstep[0]
            raw_id = fast5_data[read+'/Raw'].attrs['read_id'].decode('UTF-8')
            if raw_id not in id_count:
                    id_count[raw_id] = 1
            else:
                id_count[raw_id] += 1
    above2 = []
    for key in id_count.keys():
        num = id_count[key]
        if num >= 2 :
            above2.append(key)
    print("Number of unique ids: " + str(len(id_count.keys())))
    print("Total number of ids: " + str(sum(id_count.values())))
    print("Difference: " + str(sum(id_count.values()) - len(id_count.keys()) ) )
    print("Above2: " + str(len(above2)))



In [16]:
f5Base = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s/fast5s_original/"

## 1a-42k
print("\n*********** Flowcell 1 *********** ")
folder = f5Base + "1a-42k"
count_ids_f5(folder)

## 2b-42k
print("\n*********** Flowcell 2 *********** ")
folder = f5Base + "2b-42k"
count_ids_f5(folder)

## 3d-42k
print("\n*********** Flowcell 3 *********** ")
folder = f5Base + "3d-42k"
count_ids_f5(folder)

## all
print("\n*********** All *********** ")
folder = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s/fast5s_original_all/"
count_ids_f5(folder)


*********** Flowcell 1 *********** 
Number of unique ids: 158581
Total number of ids: 158581
Difference: 0
Above2: 0

*********** Flowcell 2 *********** 


In [13]:
# count the number of references in a blastn file
def count_ids_blast(blastnFile):
    blast_ids = {}
    with open(blastnFile, 'r') as f:
        for line in f:
            if line[0] == ">":
                content = line.strip("\n").strip().strip(">")
                read_id = content.split()[0]
                if read_id not in blast_ids:
                    blast_ids[read_id] = 1
                else:
                    blast_ids[read_id] += 1
    above2 = []
    for key in blast_ids.keys():
        num = blast_ids[key]
        if num >= 2 :
            above2.append(key)
    print("Number of unique ids: " + str(len(blast_ids.keys())))
    print("Total number of ids: " + str(sum(blast_ids.values())))
    print("Difference: " + str(sum(blast_ids.values()) - len(blast_ids.keys()) ) )
    print("Above2: " + str(len(above2)))



In [6]:
# concatenate files
blastnBase = "/media/victor/USB/MSc_basecall/Data/3XR6/ref_3xr6/blastn/"
with open(blastnBase + "fc123s_42k.out", 'w') as outfile:
    for fname in ["fc1s_42k.out", "fc2s_42k.out", "fc3s_42k.out"]:
        with open(blastnBase + fname, 'r') as readfile:
            for line in readfile:
                outfile.write(line)


with open(blastnBase + "fc123s_42k_addonly.out", 'w') as outfile:
    for fname in ["fc1s_42k_addonly.out", "fc2s_42k_addonly.out", "fc3s_42k_addonly.out"]:
        with open(blastnBase + fname, 'r') as readfile:
            for line in readfile:
                outfile.write(line)


In [14]:
blastnBase = "/media/victor/USB/MSc_basecall/Data/3XR6/ref_3xr6/blastn/"

## 1a-42k
print("\n*********** Flowcell 1 *********** ")
blastnFile = blastnBase + "fc1s_42k.out"
count_ids_blast(blastnFile)

## 2b-42k
print("\n*********** Flowcell 2 *********** ")
blastnFile = blastnBase + "fc2s_42k.out"
count_ids_blast(blastnFile)

## 3d-42k
print("\n*********** Flowcell 3 *********** ")
blastnFile = blastnBase + "fc3s_42k.out"
count_ids_blast(blastnFile)

## all
print("\n*********** All *********** ")
blastnFile = blastnBase + "fc123s_42k.out"
count_ids_blast(blastnFile)

''' NB: ADD HISTOGTRAM ?? '''




*********** Flowcell 1 *********** 
Number of unique ids: 40201
Total number of ids: 41465
Difference: 1264
Above2: 1226

*********** Flowcell 2 *********** 
Number of unique ids: 14599
Total number of ids: 14800
Difference: 201
Above2: 194

*********** Flowcell 3 *********** 
Number of unique ids: 12038
Total number of ids: 12223
Difference: 185
Above2: 181

*********** All *********** 
Number of unique ids: 66838
Total number of ids: 68488
Difference: 1650
Above2: 1601


' NB: ADD HISTOGTRAM ?? '

In [10]:
blastnBase = "/media/victor/USB/MSc_basecall/Data/3XR6/ref_3xr6/blastn/"

## 1a-42k
print("\n*********** Flowcell 1 _addonly *********** ")
blastnFile = blastnBase + "fc1s_42k_addonly.out"
count_ids_blast(blastnFile)

## 2b-42k
print("\n*********** Flowcell 2 _addonly*********** ")
blastnFile = blastnBase + "fc2s_42k_addonly.out"
count_ids_blast(blastnFile)

## 3d-42k
print("\n*********** Flowcell 3 _addonly*********** ")
blastnFile = blastnBase + "fc3s_42k_addonly.out"
count_ids_blast(blastnFile)

## all
print("\n*********** All _addonly*********** ")
blastnFile = blastnBase + "fc123s_42k_addonly.out"
count_ids_blast(blastnFile)

''' NB: ADD HISTOGTRAM ?? '''



*********** Flowcell 1 _addonly *********** 
Number of unique ids: 61517
Total number of ids: 63964
Difference: 2447
Above2: 2378

*********** Flowcell 2 _addonly*********** 
Number of unique ids: 26071
Total number of ids: 26590
Difference: 519
Above2: 512

*********** Flowcell 3 _addonly*********** 
Number of unique ids: 21511
Total number of ids: 21991
Difference: 480
Above2: 472

*********** All _addonly*********** 
Number of unique ids: 109099
Total number of ids: 112545
Difference: 3446
Above2: 3362


' NB: ADD HISTOGTRAM ?? '

#### Number of reads that align to several references

In [32]:
'''Number of reads that align to several references'''
def count_reads_with_severalRef(blastnFileList):
    id_dic = {}
    for blastnFile in blastnFileList:
        with open(blastnFile, 'r') as f:
            current_querry = ''
            for line in f:
                if line[0:6] == "Query=":
                    current_querry = line.split()[1]
                elif line[0] == '>':
                    read_id = line[1:].split()[0]
                    if read_id not in id_dic:
                        id_dic[read_id] = set([current_querry])
                    else:
                        id_dic[read_id].add(current_querry)
    count_severalRef = 0
    several_dic = set()
    for key in id_dic:
        if len(id_dic[key]) > 1:
            count_severalRef += 1
            several_dic.add(key)
    print("Number of read_ids that align to several references: " + str(count_severalRef))
    return(several_dic)


In [33]:
blastn_path = "/media/victor/USB/MSc_basecall/Data/3XR6/ref_3xr6/blastn/"
blastnFileList = [blastn_path + "fc1s_42k.out",
                  blastn_path + "fc2s_42k.out",
                   blastn_path + "fc3s_42k.out"]
                   
several_dic = count_reads_with_severalRef(blastnFileList)

Number of read_ids that align to several references: 1601


#### For each reference, how many reads align to it accross all flowcells

In [38]:

# Number of reads per rereference
def read_id_per_reference(blastnFileList):
    nb_read_id = 0
    ref_dic = {}
    for blastnFile in blastnFileList:
        with open(blastnFile, 'r') as f:
            current_querry = ''
            for line in f:
                if line[0:6] == "Query=":
                    current_querry = line.split()[1]
                    if current_querry not in ref_dic:
                        nb_read_id += 1
                        ref_dic[current_querry] = set()
                elif line[0] == '>':
                    read_id = line[1:].split()[0]
                    ref_dic[current_querry].add(read_id)

    nb_ref_sev_align = 0      
    max_num_reads = 0
    max_key = ''
    for key in ref_dic:
        if len(ref_dic[key]) > 1:
            nb_ref_sev_align += 1
        if len(ref_dic[key]) > max_num_reads:
            max_num_reads = len(ref_dic[key])
            max_key = key
    print("Number of reference with more than one read id that align to it: " + str(nb_ref_sev_align))
    print("Total unique ref id: " + str(nb_read_id))
    print("Max number of reads " + str(max_num_reads) + " for the reference " + str(max_key))
    return(ref_dic)

In [39]:
blastn_path = "/media/victor/USB/MSc_basecall/Data/3XR6/ref_3xr6/blastn/"
blastnFileList = [blastn_path + "fc1s_42k.out",
                  blastn_path + "fc2s_42k.out",
                   blastn_path + "fc3s_42k.out"]
                   
ref_dic = read_id_per_reference(blastnFileList)

Number of reference with more than one read id that align to it: 19418
Total unique ref id: 42000
Max number of reads 13 for the reference bc25mer_943


## Filtered dataset analysis

#### checks that read_ids in blastn are in fast5 files

In [131]:
# count the number of ids from blast file that are not present in the fast5 file
def count_missing_id_fast5(blastnFile, fast5Folder):
    # create dic of ids from fast5 file
    raw_ids = {}
    filenames = glob.glob(fast5Folder + "/*.fast5")
    for fast5File in filenames:
        # try to open fast5 file
        if not fast5File.endswith('fast5'):
            print("Not a fast5 file")
            return
        try:
            fast5_data = h5py.File(fast5File, 'r')
        except IOError:
            print('Error opening file')
            return
        
        read_list = list(fast5_data.items())
        for i in range(len(read_list)):
            readstep = (read_list[i])
            read = readstep[0]
            raw_id = fast5_data[read+'/Raw'].attrs['read_id'].decode('UTF-8')
            
            if raw_id not in raw_ids:
                raw_ids[raw_id] = 1
            else:
                raw_ids[raw_id] += 1
        fast5_data.close()
    # create dic of ids from blastn file
    blast_ids = {}
    with open(blastnFile, 'r') as f:
        for line in f:
            if line[0] == ">":
                content = line.strip("\n").strip().strip(">")
                read_id = content.split()[0]
                if read_id not in blast_ids:
                    blast_ids[read_id] = 1
                else:
                    blast_ids[read_id] += 1
                   
    # comput the number of missing elements
    count_differences = 0
    missing_ids = []
    for key in blast_ids:
        if key not in raw_ids:
            count_differences+=1
            missing_ids.append(key)
    print("There are: " + str(count_differences) + " missing ids")
    




In [99]:
fast5_base = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s/"
blastn_base = "/media/victor/USB/MSc_basecall/Data/3XR6/ref_3xr6/blastn/"


## 1a-42k
print("\n*********** Flowcell 1 *********** ")
fast5Folder = fast5_base + "/1a-42k/"
blastnFile = blastn_base + "fc1s_42k.out"
count_missing_id_fast5(blastnFile, fast5Folder)

## 2b-42k
print("\n*********** Flowcell 2 *********** ")
fast5Folder = fast5_base + "/2b-42k/"
blastnFile = blastn_base + "fc2s_42k.out"
count_missing_id_fast5(blastnFile, fast5Folder)

## 3d-42k
print("\n*********** Flowcell 3 *********** ")
fast5Folder = fast5_base + "/3c-42k/"
blastnFile = blastn_base + "fc3s_42k.out"
count_missing_id_fast5(blastnFile, fast5Folder)


*********** Flowcell 1 *********** 
There are: 0 missing ids

*********** Flowcell 2 *********** 
There are: 0 missing ids

*********** Flowcell 3 *********** 
There are: 0 missing ids


#### Number of read in the filtered dataset

In [137]:
# get the name of the file without path and without extension
def get_just_file_name(file_path):
    file_path = file_path.strip()
    splited_fname = file_path.split('/')
    simple_file_name = splited_fname[-1]
    simple_file_name = os.path.splitext(simple_file_name)[0]
    return(simple_file_name)



# count the number of reads fast5 files
def count_read_ids_fast5(fast5Folder):
    filenames = glob.glob(fast5Folder + "/*.fast5")
    count_id = {}
    for fast5File in filenames:
        # try to open fast5 file
        if not fast5File.endswith('fast5'):
            print("Not a fast5 file")
            return
        try:
            fast5_data = h5py.File(fast5File, 'r')
        except IOError:
            print('Error opening file')
            return
        short_file_name = get_just_file_name(fast5File)
        read_list = list(fast5_data.items())
        count_id[short_file_name] = len(read_list)
        fast5_data.close()
    return(count_id)

In [13]:
fast5base_original = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s/"
fast5base_filtered = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s_filtered_long/"

# Filtering using blastn e-value for the entire sequence
print("FILTERING WITH ENTIRE 120-LONG SEQUENCES") 

folders = ["1a-42k/", "2b-42k/", "3d-42k/"]
for i in range(3):
    print("\n*********** Flowcell " + str(i+1) + " *********** ")
    folder = folders[i]
    fast5Folder_original = fast5base_original + folder
    fast5Folder_filtered = fast5base_filtered + folder
    dic_count_original = count_read_ids_fast5(fast5Folder_original)
    dic_count_filtered = count_read_ids_fast5(fast5Folder_filtered)
    sum_original = sum(dic_count_original.values())
    sum_filtered = sum(dic_count_filtered.values())
    print("Initial number of reads: " + str(sum_original))
    print("After filtering, number of reads: " + str(sum_filtered))
    print("i.e. " + str(100 * round(sum_filtered/sum_original,4)) + "%" )



FILTERING WITH ENTIRE 120-LONG SEQUENCES

*********** Flowcell 1 *********** 
Initial number of reads: 158581
After filtering, number of reads: 40201
i.e. 25.35%

*********** Flowcell 2 *********** 
Initial number of reads: 94307
After filtering, number of reads: 14599
i.e. 15.479999999999999%

*********** Flowcell 3 *********** 
Initial number of reads: 89766
After filtering, number of reads: 12038
i.e. 13.41%


In [14]:
fast5base_original = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s/"
fast5base_filtered = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s_filtered_short/"

# Filtering using blastn e-value for the entire sequence
print("FILTERING WITH 25-LONG SEQUENCES OF INTEREST") 

folders = ["1a-42k/", "2b-42k/", "3d-42k/"]
for i in range(3):
    print("\n*********** Flowcell " + str(i+1) + " *********** ")
    folder = folders[i]
    fast5Folder_original = fast5base_original + folder
    fast5Folder_filtered = fast5base_filtered + folder
    dic_count_original = count_read_ids_fast5(fast5Folder_original)
    dic_count_filtered = count_read_ids_fast5(fast5Folder_filtered)
    sum_original = sum(dic_count_original.values())
    sum_filtered = sum(dic_count_filtered.values())
    print("Initial number of reads: " + str(sum_original))
    print("After filtering, number of reads: " + str(sum_filtered))
    print("i.e. " + str(100 * round(sum_filtered/sum_original,4)) + "%")



FILTERING WITH 25-LONG SEQUENCES OF INTEREST

*********** Flowcell 1 *********** 
Initial number of reads: 158581
After filtering, number of reads: 61517
i.e. 38.79%

*********** Flowcell 2 *********** 
Initial number of reads: 94307
After filtering, number of reads: 26071
i.e. 27.639999999999997%

*********** Flowcell 3 *********** 
Initial number of reads: 89766
After filtering, number of reads: 21511
i.e. 23.96%


## Other

#### count how many reads per fast5 file on average

In [3]:
# count how many reads in a fast5 file on average

def count_average_reads_fast5(fast5Folder):
    global_sum = 0
    filenames = glob.glob(fast5Folder + "/*.fast5")
    first_time = True
    for fast5File in filenames:
        # try to open fast5 file
        if not fast5File.endswith('fast5'):
            print("Not a fast5 file")
            return
        try:
            fast5_data = h5py.File(fast5File, 'r')
        except IOError:
            print('Error opening file')
            return
        read_list = list(fast5_data.items())
        length = len(read_list)
        if first_time:
            print(length)
            first_time = False
        global_sum += length
        fast5_data.close()
    average = global_sum / len(filenames)
    print("There are " + str(average) + " reads per fast5 file on average")
    return(average)



In [6]:
fast5base_original = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s/"
fast5base_filtered = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s_filtered_short/"

# Filtering using blastn e-value for the entire sequence
print("FILTERING WITH 25-LONG SEQUENCES OF INTEREST") 

folders = ["1a-42k/", "2b-42k/", "3d-42k/"]
for i in range(3):
    print("\n*********** Flowcell " + str(i+1) + " *********** ")
    folder = folders[i]
    fast5Folder_original = fast5base_original + folder
    fast5Folder_filtered = fast5base_filtered + folder
    print("Original")
    mean_length_original = count_average_reads_fast5(fast5Folder_original)
    print("Filtered")
    mean_length_filtered = count_average_reads_fast5(fast5Folder_filtered)
    
    


FILTERING WITH 25-LONG SEQUENCES OF INTEREST

*********** Flowcell 1 *********** 
Original
4000
There are 3964.525 reads per fast5 file on average
Filtered
1561
There are 1537.925 reads per fast5 file on average

*********** Flowcell 2 *********** 
Original
4000
There are 3929.4583333333335 reads per fast5 file on average
Filtered
1455
There are 1086.2916666666667 reads per fast5 file on average

*********** Flowcell 3 *********** 
Original
4000
There are 3902.8695652173915 reads per fast5 file on average
Filtered
60
There are 935.2608695652174 reads per fast5 file on average


#### average signal length

In [3]:
# compute average signal lenght of a 

def count_average_reads_fast5(fast5Folder):
    filenames = glob.glob(fast5Folder + "/*.fast5")
    len_track = []
    first = True
    for fast5File in filenames:
        # try to open fast5 file
        if not fast5File.endswith('fast5'):
            print("Not a fast5 file")
            return
        try:
            fast5_data = h5py.File(fast5File, 'r')
        except IOError:
            print('Error opening file')
            return
        read_list = list(fast5_data.items())
        for i in range(len(read_list)):
            readstep = (read_list[i])
            read = readstep[0]
            signal_path = read + '/Raw/Signal'
            sig = np.array(fast5_data.get(signal_path), dtype=np.float32)
            sig_len = len(sig)
            len_track.append(sig_len)
        if first:
            print("sum lengths first file: " + str( sum(len_track)) )
            first = False

        fast5_data.close()
    mean_len = np.mean(np.array(len_track))
    print("There are " + str(mean_len) + " points per signal on average")
    return(mean_len)

In [53]:
fast5base_original = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s/"
fast5base_filtered = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s_filtered_short/"

# Filtering using blastn e-value for the entire sequence
print("FILTERING WITH 25-LONG SEQUENCES OF INTEREST") 

folders = ["1a-42k/", "2b-42k/", "3d-42k/"]
for i in range(3):
    print("\n*********** Flowcell " + str(i+1) + " *********** ")
    folder = folders[i]
    fast5Folder_original = fast5base_original + folder
    fast5Folder_filtered = fast5base_filtered + folder
    print("Original")
    mean_length_original = count_average_reads_fast5(fast5Folder_original)
    print("Filtered")
    mean_length_filtered = count_average_reads_fast5(fast5Folder_filtered)
    

FILTERING WITH 25-LONG SEQUENCES OF INTEREST

*********** Flowcell 1 *********** 
Original
sum lengths first file: 19789662
There are 5183.972184561832 points per signal on average
Filtered
sum lengths first file: 6961590
There are 4532.765073069233 points per signal on average

*********** Flowcell 2 *********** 
Original
sum lengths first file: 20731984
There are 5367.725545293562 points per signal on average
Filtered
sum lengths first file: 7616893
There are 4820.364466265199 points per signal on average

*********** Flowcell 3 *********** 
Original
sum lengths first file: 20146232
There are 5600.4700220573495 points per signal on average
Filtered
sum lengths first file: 445792
There are 4861.318023336898 points per signal on average


In [7]:
# compute average signal lenght of a 

def count_average_reads_fast5_bis(fast5Folder):
    filenames = glob.glob(fast5Folder + "/*.fast5")
    len_track = []
    first = True
    print(len(filenames))
    for fast5File in filenames:
        # try to open fast5 file
        if not fast5File.endswith('fast5'):
            print("Not a fast5 file")
            return
        try:
            fast5_data = h5py.File(fast5File, 'r')
        except IOError:
            print('Error opening file')
            return
        raw_attr = fast5_data['Raw/Reads/']
        read_name = list(raw_attr.keys())[0]
        raw_signal = raw_attr[read_name + '/Signal'][()]
        sig = np.array(raw_signal, dtype=np.float32)
        sig_len = len(sig)
        len_track.append(sig_len)
        if first:
            print("sum lengths first file: " + str( sum(len_track)) )
            first = False

        fast5_data.close()
    mean_len = np.mean(np.array(len_track))
    print("There are " + str(mean_len) + " points per signal on average")
    return(mean_len)

In [8]:
print("****** natural DNA data ******" )
fast5base_original = "/media/victor/USB/MSc_basecall/Data/Klebsiella_pneumoniae_KSB1_7F/training_fast5s/0000/"

mean_length_original = count_average_reads_fast5_bis(fast5base_original)


****** natural DNA data ******
808
sum lengths first file: 112357
There are 150612.94925742573 points per signal on average


#### count how many signals inferior to 2048 points

In [54]:
# compute the number of signals with less than 2048 points

def count2048reads(fast5Folder):
    filenames = glob.glob(fast5Folder + "/*.fast5")
    inf2048 = 0
    total = 0
    for fast5File in filenames:
        # try to open fast5 file
        if not fast5File.endswith('fast5'):
            print("Not a fast5 file")
            return
        try:
            fast5_data = h5py.File(fast5File, 'r')
        except IOError:
            print('Error opening file')
            return
        read_list = list(fast5_data.items())
        for i in range(len(read_list)):
            readstep = (read_list[i])
            read = readstep[0]
            signal_path = read + '/Raw/Signal'
            sig = np.array(fast5_data.get(signal_path), dtype=np.float32)
            sig_len = len(sig)
            if(sig_len<= 2048):
                inf2048 += 1
            total += 1
        fast5_data.close()
    print("There are " + str(inf2048) + " reads that have less than 2048 files")
    print("There are " + str(total) + " reads in total")
    print("ie " + str(round(inf2048 / total, 4)*100) + "%")
    return(inf2048)

In [55]:
fast5base_original = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s/"
fast5base_filtered = "/media/victor/USB/MSc_basecall/Data/3XR6/fast5s_filtered_short/"

# Filtering using blastn e-value for the entire sequence
print("FILTERING WITH 25-LONG SEQUENCES OF INTEREST") 

folders = ["1a-42k/", "2b-42k/", "3d-42k/"]
for i in range(3):
    print("\n*********** Flowcell " + str(i+1) + " *********** ")
    folder = folders[i]
    fast5Folder_original = fast5base_original + folder
    fast5Folder_filtered = fast5base_filtered + folder
    print("Original")
    mean_length_original = count2048reads(fast5Folder_original)
    print("Filtered")
    mean_length_filtered = count2048reads(fast5Folder_filtered)

FILTERING WITH 25-LONG SEQUENCES OF INTEREST

*********** Flowcell 1 *********** 
Original
There are 2 reads that have less than 2048 files
There are 158581 reads in total
ie 0.0%
Filtered
There are 2 reads that have less than 2048 files
There are 61517 reads in total
ie 0.0%

*********** Flowcell 2 *********** 
Original
There are 0 reads that have less than 2048 files
There are 94307 reads in total
ie 0.0%
Filtered
There are 0 reads that have less than 2048 files
There are 26071 reads in total
ie 0.0%

*********** Flowcell 3 *********** 
Original
There are 0 reads that have less than 2048 files
There are 89766 reads in total
ie 0.0%
Filtered
There are 0 reads that have less than 2048 files
There are 21511 reads in total
ie 0.0%
